In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'gabor'
CHANNEL = 'gray'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_gabor_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,39241.18,1.19,-0.21,0.44,0.12,-0.15,-0.13,-1.98,0.64,-0.43,...,-0.17,0.34,-0.02,0.77,-0.07,-0.45,0.20,0.03,0.02,0.05
1,1.19,16581.21,0.62,-0.45,-0.06,-0.10,-0.01,1.37,-0.23,-0.36,...,0.00,-0.00,0.01,0.90,-0.20,-0.11,-0.29,0.06,0.05,0.07
2,-0.21,0.62,6968.56,-0.12,0.00,0.07,0.04,0.08,0.49,-0.14,...,0.04,-0.04,0.03,-0.64,0.54,-0.02,-0.34,-0.01,0.09,-0.02
3,0.44,-0.45,-0.12,2952.46,-0.05,0.03,-0.02,-0.61,-0.22,0.00,...,-0.04,-0.06,0.02,0.32,-0.04,0.13,0.06,0.04,-0.04,-0.00
4,0.12,-0.06,0.00,-0.05,1229.97,-0.01,0.01,-0.02,0.00,0.24,...,-0.00,0.01,0.01,0.01,0.13,-0.09,0.09,0.01,0.04,-0.02
5,-0.15,-0.10,0.07,0.03,-0.01,452.41,0.00,-0.27,0.09,-0.03,...,-0.02,-0.01,0.01,-0.15,0.02,0.01,0.04,-0.00,0.00,-0.00
6,-0.13,-0.01,0.04,-0.02,0.01,0.00,276.68,0.06,0.00,0.09,...,0.02,-0.00,-0.01,-0.05,-0.01,0.02,-0.03,0.03,-0.01,0.00
7,-1.98,1.37,0.08,-0.61,-0.02,-0.27,0.06,53970.94,-0.09,0.56,...,-0.02,0.15,-0.13,0.44,-1.51,-0.39,-0.56,0.32,0.17,-0.13
8,0.64,-0.23,0.49,-0.22,0.00,0.09,0.00,-0.09,22259.40,0.36,...,-0.12,0.04,0.00,0.60,0.83,-0.73,0.21,0.17,-0.04,-0.04


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00002,-0.00004,-0.00004,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00002,-0.00000,-0.00003,0.00002,0.00000,0.00001,0.00002
1,0.00005,1.00000,0.00006,-0.00006,-0.00001,-0.00004,-0.00000,0.00005,-0.00001,-0.00003,...,0.00000,-0.00000,0.00000,0.00003,-0.00001,-0.00001,-0.00004,0.00001,0.00002,0.00004
2,-0.00001,0.00006,1.00000,-0.00003,0.00000,0.00004,0.00003,0.00000,0.00004,-0.00002,...,0.00001,-0.00002,0.00003,-0.00004,0.00005,-0.00000,-0.00007,-0.00000,0.00005,-0.00001
3,0.00004,-0.00006,-0.00003,1.00000,-0.00003,0.00002,-0.00002,-0.00005,-0.00003,0.00000,...,-0.00002,-0.00005,0.00003,0.00003,-0.00001,0.00003,0.00002,0.00002,-0.00003,-0.00001
4,0.00002,-0.00001,0.00000,-0.00003,1.00000,-0.00002,0.00001,-0.00000,0.00000,0.00007,...,-0.00000,0.00002,0.00001,0.00000,0.00003,-0.00003,0.00004,0.00001,0.00005,-0.00004
5,-0.00004,-0.00004,0.00004,0.00002,-0.00002,1.00000,0.00000,-0.00005,0.00003,-0.00002,...,-0.00003,-0.00002,0.00003,-0.00003,0.00001,0.00001,0.00004,-0.00000,0.00000,-0.00001
6,-0.00004,-0.00000,0.00003,-0.00002,0.00001,0.00000,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00003,-0.00001,-0.00001,0.00001,-0.00003,0.00005,-0.00003,0.00001
7,-0.00004,0.00005,0.00000,-0.00005,-0.00000,-0.00005,0.00002,1.00000,-0.00000,0.00002,...,-0.00000,0.00003,-0.00004,0.00001,-0.00005,-0.00002,-0.00004,0.00004,0.00003,-0.00004
8,0.00002,-0.00001,0.00004,-0.00003,0.00000,0.00003,0.00000,-0.00000,1.00000,0.00002,...,-0.00002,0.00001,0.00000,0.00002,0.00004,-0.00006,0.00002,0.00003,-0.00001,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

13.506271674104834

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.96793854e+07 5.24259852e+07 4.75118726e+07 3.78654870e+07
 3.75587734e+07 3.45429084e+07 1.18065497e+07 8.54073650e+06
 7.63155137e+06 6.74384400e+06 6.60060677e+06 5.93618578e+06
 2.13987986e+06 1.40286705e+06 1.30423496e+06 1.15405610e+06
 1.06123787e+06 9.46297195e+05 3.70485577e+05 2.60678795e+05
 2.44773901e+05 2.06866642e+05 1.90486217e+05 1.62869875e+05
 6.14075826e+04 4.56048547e+04 3.70051399e+04 3.61375657e+04
 3.31823331e+04 2.58436350e+04 1.00332512e+04 6.63535515e+03
 6.10099316e+03 5.13985942e+03 4.95953827e+03 3.60341476e+03
 2.25189930e+03 1.67206747e+03 1.25708271e+03 1.12100345e+03
 1.01677126e+03 5.30669804e-24]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.030865,-0.006590,-0.002557,-0.001081,-4.445396e-04,-0.000168,-9.850052e-05,0.994376,-0.009726,-0.003531,...,-0.000412,-0.000165,-0.000075,-0.053378,-0.007313,-0.002741,-0.001188,-0.000440,-0.000184,-0.000080
1,-0.048867,-0.006673,-0.002512,-0.001060,-4.361474e-04,-0.000156,-1.003502e-04,0.052980,-0.010177,-0.003483,...,-0.000397,-0.000157,-0.000070,-0.217890,-0.007375,-0.002654,-0.001142,-0.000444,-0.000184,-0.000072
2,-0.096584,-0.009930,-0.003708,-0.001529,-6.361653e-04,-0.000237,-1.447226e-04,0.056115,-0.015380,-0.005138,...,-0.000582,-0.000236,-0.000103,0.963290,-0.011048,-0.003932,-0.001702,-0.000631,-0.000270,-0.000114
3,-0.370857,-0.001541,-0.000529,-0.000227,-9.712028e-05,-0.000035,-1.486285e-05,0.004531,-0.002495,-0.000759,...,-0.000090,-0.000043,-0.000011,0.012214,-0.001690,-0.000552,-0.000248,-0.000089,-0.000037,-0.000014
4,0.709388,0.000055,0.000006,0.000002,2.762564e-07,-0.000006,-8.856155e-07,0.000045,0.000079,-0.000015,...,-0.000003,0.000008,0.000001,0.000069,-0.000003,0.000003,0.000010,0.000005,0.000003,0.000002
5,0.587151,-0.026812,-0.009428,-0.003859,-1.603987e-03,-0.000591,-3.623433e-04,0.066676,-0.044705,-0.013242,...,-0.001493,-0.000603,-0.000261,0.142556,-0.030123,-0.010072,-0.004276,-0.001612,-0.000679,-0.000280
6,0.018870,-0.040340,-0.008110,-0.003169,-1.295212e-03,-0.000472,-2.910599e-04,0.011305,0.992102,-0.012244,...,-0.001206,-0.000486,-0.000210,0.015003,-0.055146,-0.008842,-0.003479,-0.001293,-0.000549,-0.000227
7,0.011186,-0.075423,-0.007870,-0.002931,-1.223360e-03,-0.000441,-2.686177e-04,0.007132,0.052324,-0.012438,...,-0.001129,-0.000441,-0.000198,0.009288,-0.199828,-0.008513,-0.003251,-0.001203,-0.000502,-0.000213
8,0.011645,-0.159447,-0.009562,-0.003534,-1.435774e-03,-0.000527,-3.232724e-04,0.007606,0.044270,-0.015585,...,-0.001337,-0.000531,-0.000239,0.009685,0.948641,-0.010519,-0.003909,-0.001451,-0.000604,-0.000252
9,0.001880,-0.611054,-0.001763,-0.000605,-2.432416e-04,-0.000089,-5.818139e-05,0.001187,0.005746,-0.002867,...,-0.000237,-0.000090,-0.000041,0.001525,0.023313,-0.001950,-0.000674,-0.000254,-0.000105,-0.000048


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.005623845810334371,
 0.029417361661789876,
 0.036709802314139006,
 0.15237926817031489,
 0.29061156742525385,
 0.412566460169302,
 0.007897646358434929,
 0.03054308289316865,
 0.051359105745791056,
 0.21532152450137398,
 0.2700645258636327,
 0.34382098679994877,
 0.008949850740203491,
 0.07143543347277359,
 0.10348345323693708,
 0.10113662061656015,
 0.28262693348033807,
 0.35318809026446085,
 0.01562765506716668,
 0.1952014427897466,
 0.23885445370995373,
 0.11965549918167939,
 0.2687746018319107,
 0.3151384988271768,
 0.025314656943897895,
 0.04841535968724331,
 0.25409711462085727,
 0.257287934969751,
 0.2623897023246927,
 0.3513241485633146,
 0.035647356377384365,
 0.25490096749413604,
 0.34368725844375103,
 0.1845624167993387,
 0.26016337133964185,
 0.5280984927101976,
 0.15730203978637458,
 0.19414258058199507,
 0.1886950967203963,
 0.29226345688062005,
 0.13960221576154996,
 0.5753689721388716]